In [ ]:
#!/usr/bin/env python3
"""
live_pip_gated_trader.py   –   final version
────────────────────────────────────────────
• Trader (price PIPs) suggests positions each bar.
• Gate watches the trader’s **always-on cumulative PnL** and runs the same
  causal zig-zag rule (threshold = `PIP_THR`).
• If the current swing (observed up to bar t-1) is *up* → trader is ON
  (GREEN); if swing is *down* or not yet defined → trader is OFF (RED).
• Plots per instrument:
      1. Price (green/red shading = gate status)
      2. Gated cumulative PnL (black)  vs  always-on PnL (grey dashed)
      3. Always-on PnL with PIP markers.
"""

# ───────────────────────── imports ────────────────────────────────────
from pathlib import Path
from typing  import List, Optional, Tuple
import numpy as np
import matplotlib.pyplot as plt

# ───────────────────────── hyper-parameters ───────────────────────────
POSITION_LIMIT   = 10_000
COMMISSION_RATE  = 0.0005
PIP_THR          = 0.30      # 30 % swing on cum PnL ⇒ new pivot & gate flip
EPS              = 1e-8

BEST_N = [200,200,400,200,400,400,100,300,100,200,
          100,500,100,300,200,100,200,100,200,300,
          100,200,500,100,400,100,100,400,100,100,
          400,500,300,200,400,500,100,400,100,100,
          100,300,400,500,100,400,500,100,500,300]
BEST_W = [10,10,100,10,100,10,100,25,10,100,
          100,50,25,10,50,25,10,25,50,100,
          100,10,10,50,25,25,10,50,25,100,
          25,10,100,50,25,10,50,100,50,25,
          25,10,100,100,50,50,10,25,25,25]

THR_GRID = np.arange(0.0, 0.700 + 0.0005, 0.0005)

# ───────────────────────── helpers ────────────────────────────────────
def safe_rel_move(a: float, b: float, eps: float = EPS) -> float:
    return (a - b) / max(abs(b), eps)

def pip_price_update(px: float, last_ext: float,
                     direction: Optional[int], thr: float) -> Tuple[int, float]:
    if direction in (0, None):
        mv = (px - last_ext) / last_ext
        if abs(mv) >= thr:
            return (1 if mv > 0 else -1), px
        return 0, last_ext
    if direction == 1:
        if px > last_ext: return 1, px
        if (last_ext - px) / last_ext >= thr: return -1, px
        return 1, last_ext
    # direction == -1
    if px < last_ext:  return -1, px
    if (px - last_ext) / last_ext >= thr: return 1, px
    return -1, last_ext

# ───────────────────────── trader ─────────────────────────────────────
class CausalPIPTrader:
    class _S:
        __slots__ = ("n","w","next_cal","thr","dir","last_ext")
        def __init__(self,n:int,w:int):
            self.n,self.w = n,w
            self.next_cal = n
            self.thr = None
            self.dir = 0
            self.last_ext = None

    def __init__(self):
        self._s = [self._S(n,w) for n,w in zip(BEST_N,BEST_W)]

    def _best_thr(self, win: np.ndarray) -> float:
        best_thr,best_pnl = THR_GRID[0], -np.inf
        for thr in THR_GRID:
            dvec = np.zeros(len(win), np.int8)
            le,d = win[0],0
            for k in range(1,len(win)):
                d,le = pip_price_update(win[k],le,d,thr)
                dvec[k] = d
            pnl = np.sum(dvec[:-1].astype(np.int32)*POSITION_LIMIT*np.diff(win))
            if pnl > best_pnl:
                best_pnl,best_thr = pnl,thr
        return best_thr

    def Alg(self, price_hist: np.ndarray) -> np.ndarray:
        nInst, t_seen = price_hist.shape
        pos = np.zeros(nInst, np.int32)
        for i in range(nInst):
            st, px = self._s[i], price_hist[i,-1]
            if st.last_ext is None: st.last_ext = px
            if t_seen >= st.next_cal:
                st.thr = self._best_thr(price_hist[i,t_seen-st.n:t_seen])
                st.next_cal += st.w
            if st.thr is None: continue
            st.dir, st.last_ext = pip_price_update(px, st.last_ext, st.dir, st.thr)
            pos[i] = st.dir * POSITION_LIMIT
        return pos

# ───────────────────────── live PIP-gate (per instrument) ─────────────
class PipGate:
    __slots__ = ("direction","last_ext","pip_idx","pip_val")
    def __init__(self):
        self.direction: Optional[str] = None
        self.last_ext = 0.0
        self.pip_idx  = [0]
        self.pip_val  = [0.0]

    def update(self, idx: int, pnl_now: float):
        mv = safe_rel_move(pnl_now, self.last_ext)
        if self.direction is None:
            if abs(mv) >= PIP_THR:
                self.direction = 'up' if mv>0 else 'down'
                self.last_ext  = pnl_now
                self.pip_idx.append(idx)
                self.pip_val.append(pnl_now)
            return

        if self.direction == 'up':
            if pnl_now > self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) <= -PIP_THR:
                self.direction = 'down'
                self.pip_idx.append(idx)
                self.pip_val.append(self.last_ext)
                self.last_ext = pnl_now
        else:  # down
            if pnl_now < self.last_ext:
                self.last_ext = pnl_now
            elif safe_rel_move(pnl_now, self.last_ext) >= PIP_THR:
                self.direction = 'up'
                self.pip_idx.append(idx)
                self.pip_val.append(self.last_ext)
                self.last_ext = pnl_now

    @property
    def allow_trade(self) -> bool:
        return self.direction == 'up'          # green only when swing is up

# ───────────────────────── main simulation ────────────────────────────
prices = np.loadtxt(Path("prices.txt"))    # (T, 50)
T, nInst = prices.shape

trader = CausalPIPTrader()
gates   = [PipGate() for _ in range(nInst)]

pos_sugg  = np.zeros((T,nInst), int)
pos_real  = np.zeros((T,nInst), int)
allow_flag= np.zeros((T,nInst), bool)

cum_sugg  = np.zeros((T,nInst), float)
cum_real  = np.zeros((T,nInst), float)

prev_sugg = np.zeros(nInst,int)
prev_real = np.zeros(nInst,int)

for t in range(T):
    # 1. Gate looks at always-on cum PnL up to t-1
    if t>0:
        for i in range(nInst):
            gates[i].update(t-1, cum_sugg[t-1,i])

    # 2. Trader suggestion
    sugg = trader.Alg(prices[:t+1].T)
    pos_sugg[t] = sugg

    # 3. Gate decision for bar t
    allow = np.array([g.allow_trade for g in gates], bool)
    allow_flag[t] = allow
    real_pos = np.where(allow, sugg, 0)
    pos_real[t] = real_pos

    # 4. PnL bookkeeping (from previous positions)
    if t>0:
        diff = prices[t]-prices[t-1]

        pnl_s = prev_sugg * diff - COMMISSION_RATE*np.abs(sugg-prev_sugg)*prices[t]
        cum_sugg[t] = cum_sugg[t-1] + pnl_s
        prev_sugg = sugg

        pnl_r = prev_real * diff - COMMISSION_RATE*np.abs(real_pos-prev_real)*prices[t]
        cum_real[t] = cum_real[t-1] + pnl_r
        prev_real = real_pos

print("Finished live run.")

# ───────────────────────── plotting ───────────────────────────────────
times = np.arange(T)
for i in range(nInst):
    fig,(ax_price,ax_pnl,ax_pip) = plt.subplots(
        3,1,sharex=True,figsize=(10,12),
        gridspec_kw={"height_ratios":[1,1,1]}
    )

    # Price with gate shading
    ax_price.plot(times, prices[:,i], lw=1, color='black')
    ax_price.fill_between(times, prices[:,i].min(), prices[:,i].max(),
                          where=allow_flag[:,i], color='green', alpha=0.18, step='mid')
    ax_price.fill_between(times, prices[:,i].min(), prices[:,i].max(),
                          where=~allow_flag[:,i], color='red', alpha=0.18, step='mid')
    ax_price.set_ylabel("Price")

    # Gated PnL
    ax_pnl.plot(times, cum_real[:,i], lw=1.2, color='black', label='Gated PnL')
    ax_pnl.axhline(0,lw=0.5)
    ax_pnl.set_ylabel("Gated PnL")
    ax_pnl.legend(frameon=False, fontsize='small')

    # Always-on PnL + PIPs
    ax_pip.plot(times, cum_sugg[:,i], lw=1.2, ls='--', color='grey',
                label='Always-on PnL')
    ax_pip.scatter(gates[i].pip_idx, gates[i].pip_val,
                   s=45, color='blue', label='PIPs')
    ax_pip.axhline(0,lw=0.5)
    ax_pip.set_ylabel("PIP PnL")
    ax_pip.set_xlabel("Timestep")
    ax_pip.legend(frameon=False, fontsize='small')

    fig.suptitle(f"Instrument {i:02d} — gate = PIP swing (delay 1)")
    plt.tight_layout(rect=[0,0.03,1,0.95])
    plt.show()
